# BladeRF Delayed Feedback Reservoir Demo

## Import Statements

In [37]:
# package with system tools
import sys

# package with os and file I/O tools
import os

# package with tools to parse .ini file
from configparser         import ConfigParser

# package with tools to interface with bladerf
from bladerf              import _bladerf

## BladeRF Device Functions

### Close the Device and Exit

In [38]:
# =============================================================================
# Close the device and exit
# =============================================================================
def shutdown( error = 0, board = None ):
    print( "Shutting down with error code: " + str(error) )
    if( board != None ):
        board.close()
    sys.exit(error)

### Print BladeRF Version Information

In [39]:
# =============================================================================
# Version information
# =============================================================================
def print_versions( device = None ):
    print( "libbladeRF version: " + str(_bladerf.version()) )
    if( device != None ):
        try:
            print( "Firmware version: " + str(device.get_fw_version()) )
        except:
            print( "Firmware version: " + "ERROR" )
            raise

        try:
            print( "FPGA version: "     + str(device.get_fpga_version()) )
        except:
            print( "FPGA version: "     + "ERROR" )
            raise

    return 0

### Autodetect Connected BladeRF Board

In [40]:
# =============================================================================
# Search for a bladeRF device attached to the host system
# Returns a bladeRF device handle.
# =============================================================================
def probe_bladerf():
    device = None
    print( "Searching for bladeRF devices..." )
    try:
        devinfos = _bladerf.get_device_list()
        if( len(devinfos) == 1 ):
            device = "{backend}:device={usb_bus}:{usb_addr}".format(**devinfos[0]._asdict())
            print( "Found bladeRF device: " + str(device) )
        if( len(devinfos) > 1 ):
            print( "Unsupported feature: more than one bladeRFs detected." )
            print( "\n".join([str(devinfo) for devinfo in devinfos]) )
            shutdown( error = -1, board = None )
    except _bladerf.BladeRFError:
        print( "No bladeRF devices found." )
        pass

    return device

### Load BladeRF FPGA with Custom Image

In [41]:
# =============================================================================
# Load FPGA
# =============================================================================
def load_fpga( device, image ):

    image = os.path.abspath( image )

    if( not os.path.exists(image) ):
        print( "FPGA image does not exist: " + str(image) )
        return -1

    try:
        print( "Loading FPGA image: " + str(image ) )
        device.load_fpga( image )
        fpga_loaded  = device.is_fpga_configured()
        fpga_version = device.get_fpga_version()

        if( fpga_loaded ):
            print( "FPGA successfully loaded. Version: " + str(fpga_version) )

    except _bladerf.BladeRFError:
        print( "Error loading FPGA." )
        raise

    return 0

In [42]:
# =============================================================================
# Load Configuration
# =============================================================================

config = ConfigParser()
config.read('txrx.ini')

# Set libbladeRF verbosity level
verbosity = config.get('common', 'libbladerf_verbosity').upper()
if  ( verbosity == "VERBOSE" ):  _bladerf.set_verbosity( 0 )
elif( verbosity == "DEBUG" ):    _bladerf.set_verbosity( 1 )
elif( verbosity == "INFO" ):     _bladerf.set_verbosity( 2 )
elif( verbosity == "WARNING" ):  _bladerf.set_verbosity( 3 )
elif( verbosity == "ERROR" ):    _bladerf.set_verbosity( 4 )
elif( verbosity == "CRITICAL" ): _bladerf.set_verbosity( 5 )
elif( verbosity == "SILENT" ):   _bladerf.set_verbosity( 6 )
else:
    print( "Invalid libbladerf_verbosity specified in configuration file:",
           verbosity )
    shutdown( error = -1, board = None )

## BladeRF DFR Test

In [43]:
# =============================================================================
# BEGIN !!!
# =============================================================================

# auto detect connected bladerf board
uut = probe_bladerf()

# if a board was not detected
if( uut == None ):
    print( "No bladeRFs detected. Exiting." )
    shutdown( error = -1, board = None )

# get connected board information
b          = _bladerf.BladeRF( uut )
board_name = b.board_name
fpga_size  = b.fpga_size


Searching for bladeRF devices...
Found bladeRF device: libusb:device=4:2


### Loading FPGA

Alternatively:
```
bladeRF-cli -l /home/oshears/Documents/bladeRF/bladerf_dfr_accelerator/python/blade_rf_demo/hostedxA4_dfr.rbf
```

In [44]:
# load bladerf fpga with custom image
if( config.getboolean(board_name + '-load-fpga', 'enable') ):
    print( "Loading FPGA..." )
    try:
        status = load_fpga( b, config.get(board_name + '-load-fpga','image_' + str(fpga_size) + 'kle' ) )
        # print("Successfully loaded FPGA.")
        # status = 0
    except:
        print( "ERROR loading FPGA." )
        raise

    if( status < 0 ):
        print( "ERROR loading FPGA." )
        shutdown( error = status, board = b )
else:
    print( "Skipping FPGA load due to configuration setting." )

status = print_versions( device = b )

Skipping FPGA load due to configuration setting.
libbladeRF version: v2.4.1 ("2.4.1-git-5a146b2a")
Firmware version: v2.4.0 ("2.4.0-git-a3d5c55f")
FPGA version: v0.14.0 ("0.14.0")


In [45]:
s = "bladerf2-rx"
rx_ch   = _bladerf.CHANNEL_RX(config.getint(s, 'rx_channel'))
rx_freq = int(config.getfloat(s, 'rx_frequency'))
rx_rate = int(config.getfloat(s, 'rx_samplerate'))
rx_gain = int(config.getfloat(s, 'rx_gain'))
rx_ns   = int(config.getfloat(s, 'rx_num_samples'))
rx_file = config.get(s, 'rx_file')

In [46]:
status = 0
num_samples = 1024

if( b == None ):
    print( "RX: Invalid device handle." )

if( rx_ch == None ):
    print( "RX: Invalid channel." )

# Configure BladeRF
ch             = b.Channel(rx_ch)
ch.frequency   = rx_freq
ch.sample_rate = rx_rate
ch.gain        = rx_gain

# Setup synchronous stream
b.sync_config(  layout         = _bladerf.ChannelLayout.RX_X1,
                fmt            = _bladerf.Format.SC16_Q11,
                num_buffers    = 16,
                buffer_size    = 8192,
                num_transfers  = 8,
                stream_timeout = 3500)

# Enable module
print( "RX: Start" )
ch.enable = True

# Create receive buffer
bytes_per_sample = 4
buf = bytearray(1024*bytes_per_sample)
num_samples_read = 0

RX: Start


In [47]:
# Save the samples
with open(rx_file, 'w') as outfile:
    while True:
        if num_samples > 0 and num_samples_read == num_samples:
            break
        elif num_samples > 0:
            num = min(len(buf)//bytes_per_sample,
                        num_samples-num_samples_read)
        else:
            num = len(buf)//bytes_per_sample

        # Read into buffer
        b.sync_rx(buf, num)
        num_samples_read += num

        # write to .txt file
        for sample_num in range(num_samples):
            sample = buf[4*sample_num:4*(sample_num+1)]
            sample.reverse()
            sample_int = (sample[0] << 24) + (sample[1] << 16) + (sample[2] << 8) + (sample[3])
            sample_str = '{:032b}'.format(sample_int)
            outfile.write(sample_str + "\n")

# Disable module
print( "RX: Stop" )
ch.enable = False

print( "RX: Done" )

RX: Stop
RX: Done


In [48]:
# close connection to board
b.close()